In [1]:
import win32com.client
import pythoncom
import pandas as pd
import time
 
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 999999)
 
 
# 로그인 이벤트 클래스
class XASessionEventHandler:
    login_state = 0
 
    def OnLogin(self, code, msg):
        if code == "0000":
            XASessionEventHandler.login_state = 1
 
# TR 이벤트 클래스
class XAQueryEventHandlerTR:
    query_state = 0
 
    def OnReceiveData(self, code):
        XAQueryEventHandlerTR.query_state = 1
 
 
class Main:
    def __init__(self):
        # TRHandler 등록
        self.instXAQueryTR = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerTR)
        # 조회횟수 변수선언
        self.req_count = 0
 
    # 로그인 함수
    def connect(self):
        id = "ssuyan"
        passwd = "tndus1!!"
        cert_passwd = "qkrtndus1!!"
 
        instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEventHandler)
        instXASession.ConnectServer("demo.ebestsec.co.kr", 20001)
        instXASession.Login(id, passwd, cert_passwd, 0, 0)
 
        while XASessionEventHandler.login_state == 0:
            pythoncom.PumpWaitingMessages()
 
    # TR.Request 함수
    def tr_request(self, req_con):
        time.sleep(0.2)
        XAQueryEventHandlerTR.query_state = 0
        self.instXAQueryTR.Request(req_con)
 
        while self.instXAQueryTR.query_state == 0:
            pythoncom.PumpWaitingMessages()
 
        self.req_count = self.req_count + 1
 
    # 주식차트(일주월)
    def req_t8413(self, shcode, gubun, qrycnt, sdate, edate, cts_date, comp_yn):
        # outblock 정의
        outblock = {'단축코드': 'shcode', '전일시가': 'jisiga', '전일고가': 'jihigh', '전일저가': 'jilow', '전일종가': 'jiclose', '전일거래량': 'jivolume',
                    '당일시가': 'disiga', '당일고가': 'dihigh', '당일저가': 'dilow', '당일종가': 'diclose', '상한가': 'highend', '하한가': 'lowend',
                    '연속일자': 'cts_date', '장시작시간(HHMMSS)': 's_time', '장종료시간(HHMMSS)': 'e_time', '동시호가처리시간(MM:분)': 'dshmin', '레코드카운트': 'rec_count'}
        outblock1 = {'날짜': 'date', '시가': 'open', '고가': 'high', '저가': 'low', '종가': 'close', '거래량': 'jdiff_vol', '거래대금': 'value',
                     '수정구분': 'jongchk', '수정비율': 'rate', '수정주가반영항목': 'pricechk', '수정비율반영거래대금': 'ratevalue', '종가등락구분': 'sign'}
 
        # 0. 변수정의
        result = {}  # OHLCV Data
 
        # 1. res파일 지정
        self.instXAQueryTR.ResFileName = 'C:/eBEST/xingAPI/Res/t8413.res'
        # 2. 데이터 입력
        self.instXAQueryTR.SetFieldData("t8413InBlock", "shcode", 0, shcode)
        self.instXAQueryTR.SetFieldData("t8413InBlock", "gubun", 0, gubun)
        self.instXAQueryTR.SetFieldData("t8413InBlock", "qrycnt", 0, qrycnt)
        self.instXAQueryTR.SetFieldData("t8413InBlock", "sdate", 0, sdate)
        self.instXAQueryTR.SetFieldData("t8413InBlock", "edate", 0, edate)
        self.instXAQueryTR.SetFieldData("t8413InBlock", "cts_date", 0, cts_date)
        self.instXAQueryTR.SetFieldData("t8413InBlock", "comp_yn", 0, comp_yn)
 
        # 3. 서버전송 - 최초 조회시 0
        self.tr_request(0)
 
        # 4. OutBlock 데이터 가져오기
        ret_outblock = {}
        for item in outblock:
            ret_outblock[item] = self.instXAQueryTR.GetFieldData("t8413OutBlock", outblock[item], 0)
        print(ret_outblock)
 
        # cts변수 업데이트
        cts_date = ret_outblock['연속일자']
        if cts_date == '':
            con_req = False
        else:
            con_req = True
 
        # 4.1 OutBlock1 데이터 가져오기
        # OutBlock1 압축해제
        self.instXAQueryTR.Decompress("t8413OutBlock1")
        
        item_count = self.instXAQueryTR.GetBlockCount("t8413OutBlock1")
        for i in range(item_count):  # 조회건수 만큼 반복하여 조회
            temp_dict = {}
            for item in outblock1:
                temp_dict[item] = self.instXAQueryTR.GetFieldData("t8413OutBlock1", outblock1[item], i)
            result[len(result)] = temp_dict
 
        # 연속조회 시작 -------------------------------------------------------------------------------------------------
        while con_req:
            # 2. 데이터 입력
            self.instXAQueryTR.SetFieldData("t8413InBlock", "cts_date", 0, cts_date)
 
            # 3. 서버전송 - 연속조회일 경우 1
            self.tr_request(1)
 
            # 4. OutBlock 데이터 가져오기
            ret_outblock = {}
            for item in outblock:
                ret_outblock[item] = self.instXAQueryTR.GetFieldData("t8413OutBlock", outblock[item], 0)
            print(ret_outblock)
 
            # cts변수 업데이트
            cts_date = ret_outblock['연속일자']
            if cts_date == '':  # cts_data가 비어있을 경우 연속조회 중지를 위해 con_req를 False로 바꿈
                con_req = False
            else:
                con_req = True
 
            # 4.1 OutBlock1 데이터 가져오기
            # OutBlock1 압축해제
            self.instXAQueryTR.Decompress("t8413OutBlock1")
 
            item_count = self.instXAQueryTR.GetBlockCount("t8413OutBlock1")
            for i in range(item_count):  # 조회건수 만큼 반복하여 조회
                temp_dict = {}
                for item in outblock1:
                    temp_dict[item] = self.instXAQueryTR.GetFieldData("t8413OutBlock1", outblock1[item], i)
                result[len(result)] = temp_dict
 
        # 최종 dictionary 결과물과 dictionary를 dataframe으로 변환한 결과물 출력
        print('조회데이터 출력 ----------------------------------------------------------------------------------------')
        print(result)
        
        print('DataFrame 출력 ---------------------------------------------------------------------------------------')
        df = pd.DataFrame(result).transpose()
        df = df.sort_values(by='날짜')
        df = df.reset_index(drop=True)
        print(df)
        print('DataFrame 저장 ---------------------------------------------------------------------------------------')
        df.to_csv(f'{shcode}_t8413.csv')
 
 
if __name__ == "__main__":
    main = Main()
    main.connect()
 
    # 일봉업데이트
    shcode = '005930'
    gubun = '2'
    qrycnt = 2000
    sdate = '19000101'
    edate = '20220707'
    cts_date = ''
    comp_yn = 'Y'
    main.req_t8413(shcode, gubun, qrycnt, sdate, edate, cts_date, comp_yn)

{'단축코드': '005930', '전일시가': '70700', '전일고가': '71400', '전일저가': '70400', '전일종가': '71400', '전일거래량': '11535955', '당일시가': '70700', '당일고가': '71200', '당일저가': '70400', '당일종가': '70500', '상한가': '92800', '하한가': '50000', '연속일자': '20140520', '장시작시간(HHMMSS)': '090000', '장종료시간(HHMMSS)': '153000', '동시호가처리시간(MM:분)': '10', '레코드카운트': '2000'}
{'단축코드': '005930', '전일시가': '70700', '전일고가': '71400', '전일저가': '70400', '전일종가': '71400', '전일거래량': '11535955', '당일시가': '70700', '당일고가': '71200', '당일저가': '70400', '당일종가': '70500', '상한가': '92800', '하한가': '50000', '연속일자': '20060427', '장시작시간(HHMMSS)': '090000', '장종료시간(HHMMSS)': '153000', '동시호가처리시간(MM:분)': '10', '레코드카운트': '2000'}
{'단축코드': '005930', '전일시가': '70700', '전일고가': '71400', '전일저가': '70400', '전일종가': '71400', '전일거래량': '11535955', '당일시가': '70700', '당일고가': '71200', '당일저가': '70400', '당일종가': '70500', '상한가': '92800', '하한가': '50000', '연속일자': '19980502', '장시작시간(HHMMSS)': '090000', '장종료시간(HHMMSS)': '153000', '동시호가처리시간(MM:분)': '10', '레코드카운트': '2000'}
{'단축코드': '005930', '전일시가': '